In [1]:
import pandas as pd
import numpy as np
import os
try:
    import cPickle as pickle
except:
    import pickle
import json
import itertools
import time

In [7]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras.models import Model,load_model
from keras.models import model_from_json
from keras.callbacks import EarlyStopping
import keras.backend as K

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\Users\gouzm\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\gouzm\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\gouzm\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\gouzm\Anaconda3\lib\imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\gouzm\Anaconda3\lib\imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Le module spécifié est introuvable.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for some

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
def buildGRU(drop, nbneurones, lag, nb_features, nb_features_add):
    vals=Input(shape=(lag, nb_features,))
    yc=Input(shape=(nb_features_add,))
    
    #xc=Input(shape=(lag_window,len_context,))
    #yc=Input(shape=(len_context,))
    
    #drop = Dropout(drop)(vals)
    #gru  = []
    #xh   = [] 
    #x    = []
    #xp   = []
    #xpr  = []
    #pred = []

    #for i in range(len_time_series):
    #    xp.append(Lambda(lambda x:x[:,:,i])(drop)) #add values dropouted (drop) and lambda filter the station
    #    xpr.append(Reshape((lag_window,1))(xp[i])) #reshape dropouted values
    #    x.append(concatenate([xc,xpr[i]])) #add context and values
    #    #add gru layer with input=x
    #    gru.append(CuDNNGRU(nbneurones, return_sequences=False, input_shape=(lag_window, len_context+1))(x[i]))
    #    xh.append(concatenate([yc,gru[i]])) # add context plus gru output
    #    pred.append(Dense(1,activation='softplus')(xh[i]))
        
    drop = Dropout(drop)(vals)
    gru = CuDNNGRU(nbneurones, return_sequences=False, input_shape=(lag,))(drop)
    #gru = LSTM(nbneurones, return_sequences=False, input_shape=(lag,))(drop)
    gruyc = concatenate([yc,gru])
    pred = Dense(1,activation='sigmoid')(gruyc)
    
    m = Model(inputs=[vals, yc], outputs=pred)
    #predc=concatenate(pred)
    #m = Model(inputs=[vals,xc,yc],outputs=predc)
    
    return m

In [5]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [6]:
#config
drop = 0.1
nbneurones = 100
config = [drop, nbneurones]
epochs=200
batch_size = 32


lag = 20
nb_features = 11
nb_features_add = 1

m = buildGRU(config[0], config[1], lag, nb_features, nb_features_add)
#m.compile(optimizer='adam', loss='mae', metrics=['mse','mae'], sample_weight_mode=None)
m.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1], sample_weight_mode=None)


NameError: name 'Input' is not defined

In [7]:
#X = np.random.random((10000,20,14))
#Y = np.random.randint(2, size=10000)
#yc = np.random.random((10000))

X = np.load("dataLSTM.npy")
Y = np.load("Y_LSTM.npy")
yc = np.load("angleLSTM.npy")

In [8]:
test_size = 0.2
Xtrain, Xval, Ytrain, Yval = train_test_split(X,Y,test_size=test_size,shuffle=True,random_state=0,stratify=Y)
yctrain, ycval, Ytrain, Yval = train_test_split(yc,Y,test_size=test_size,shuffle=True,random_state=0,stratify=Y)

In [9]:
m.fit([Xtrain,yctrain], Ytrain, epochs=epochs, validation_data=([Xval, ycval], Yval), batch_size=batch_size)

Instructions for updating:
Use tf.cast instead.
Train on 107250 samples, validate on 26813 samples
Epoch 1/200


InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' used by node cu_dnngru_1/CudnnRNN (defined at C:\Users\gouzm\Anaconda3\lib\site-packages\keras\layers\cudnn_recurrent.py:297) with these attrs: [is_training=true, seed2=0, input_mode="linear_input", T=DT_FLOAT, dropout=0, rnn_mode="gru", direction="unidirectional", seed=87654321]
Registered devices: [CPU]
Registered kernels:
  <no registered kernels>

	 [[node cu_dnngru_1/CudnnRNN (defined at C:\Users\gouzm\Anaconda3\lib\site-packages\keras\layers\cudnn_recurrent.py:297) ]]

Caused by op 'cu_dnngru_1/CudnnRNN', defined at:
  File "C:\Users\gouzm\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\gouzm\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\gouzm\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\gouzm\Anaconda3\lib\site-packages\traitlets\confi

In [ ]:
list_drop = [0.01, 0.001]
list_neurones = [100,200,300]
list_epoch = [25, 50, 100 ]

batch_size=128
lag_window = 100

n_split = 5
val_size=0.2

In [ ]:
#Optimization
CvGridResLoc = []
for rs in range(n_split):
    print ("split ",rs+1)
    Xvapp,Xvval,Yvapp,Yvval=None,None,None,None
    Xcapp,Xcval,Ycapp,Ycval=None,None,None,None
    Xvapp,Xvval,Yvapp,Yvval = train_test_split(XvTrain,YvTrain,test_size=val_size, shuffle=True,random_state=rs)
    Xcapp,Xcval,Ycapp,Ycval = train_test_split(XcTrain,YcTrain,test_size=val_size, shuffle=True,random_state=rs)
    #iapp =  int(np.floor(XvTrain.shape[0]*(1-((rs+1)*0.1))))
    #iplus = int(np.floor(XvTrain.shape[0]*0.1))+1
    #Xvapp,Xvval,Yvapp,Yvval = XvTrain[:iapp],XvTrain[iapp:],YvTrain[:iapp],YvTrain[iapp:]
    #Xcapp,Xcval,Ycapp,Ycval = XcTrain[:iapp],XcTrain[iapp:],YcTrain[:iapp],YcTrain[iapp:]
    
    GridResLoc = {}
    cur=1
    for config in itertools.product(*hyperparams):
        t=time.time()
        print ("model ",cur,", config: ",config)
        cur+=1
        m = buildGRUUNI(config[0],config[1], my_model["lag_window"],
                        len(my_model["time_series"]),Ycapp.shape[1])
        m.compile(optimizer='adam',loss='mae',metrics=['mse','mae'],sample_weight_mode=None)
        w = np.linspace(1,config[2],XvTrain.shape[0])
        #w = np.reshape(np.repeat(np.flip(w,0),my_model['lag_window']),(w.shape[0],my_model['lag_window']))

        #w = np.reshape(np.repeat(np.flip(w,0),my_model["lag_window"]),(w.shape[0],my_model["lag_window"]))
        wapp,wval = train_test_split(w,test_size=val_size,shuffle=True,random_state=rs)
        #wapp,wval = w[:iapp],w[iapp:]

        h = m.fit([Xvapp,Xcapp,Ycapp],Yvapp,epochs=max(list_epoch),
                  validation_data=([Xvval,Xcval,Ycval], Yvval),sample_weight=wapp,batch_size=batch_size)
        
        for nbe in list_epoch:
            GridResLoc[config +(nbe,)]={'history':h.history,'mae':h.history['val_loss'][nbe-1]}
        print ("time: ",time.time()-t)
    CvGridResLoc.append(GridResLoc)